<a href="https://colab.research.google.com/github/dhkimxx/bigData-analysis-examples/blob/main/analysisExample2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
x_train = pd.read_csv('titanic_x_train.csv', encoding = 'CP949')
x_test= pd.read_csv('titanic_x_test.csv', encoding = 'CP949')
y_train = pd.read_csv('titanic_y_train.csv', encoding = 'CP949')

data = pd.concat([x_train, y_train], axis = 1)

x_test_passenger_id = x_test['PassengerId']
x_train = x_train.drop(columns = ['PassengerId', '티켓번호','승객이름', '나이', '객실번호'])
x_test = x_test.drop(columns = ['PassengerId', '티켓번호','승객이름', '나이', '객실번호'])
y_train = pd.DataFrame(y_train.iloc[:,1], columns = ['Survived'])

x_train = x_train.fillna('S')
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first = True).rename(columns = {'Q': '선착장Q', 'S': '선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis = 1).drop(columns = '선착장')
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first = True).rename(columns = {'Q': '선착장Q', 'S': '선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis = 1).drop(columns = '선착장')

x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns = ['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns = ['형제자매배우자수', '부모자식수'])

from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size = 0.2)
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators = 100, max_depth = 5, eval_metric = 'error', random_state = 10)
model.fit(X_TRAIN, Y_TRAIN)

y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns = {0:'Survived'})
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

from sklearn.metrics import roc_auc_score
#print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

final = pd.concat([x_test_passenger_id, y_test_predicted], axis = 1)
final.to_csv('12345.csv', index = False)
final = pd.read_csv('12345.csv')
print(final)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
